# Calculate Q-Rookley

- Input data "20172022_processed_1_3_4.csv", from 2017/7/1 to 2022/12/31, same as used in Figlewski

- Use 500 rolling window for P density (when calculating EPK)

- Use logreturn instead of log+1

- Use 
```
r.source('Q_Rookley_main_3.R')
```
and
```
'EPK_library_1.r'
```
to calculate Q_Rookley, along with 
```
get_btc_prices_2015_2022
```
for physical density.

- Interest rate: 0

- Moneyness range is [min moneyness, max moneyness]

- Bandwidth 0.3

- tau=47

In [3]:
import platform
plat = platform.system()
import locale
if plat == 'Darwin':
    print('Using Macos Locale')
    locale.setlocale(locale.LC_ALL, 'en_US.UTF-8')
else:
    locale.setlocale(locale.LC_ALL, 'en_US.utf8')
import datetime
import pandas as pd
import sys
import matplotlib.pyplot as plt
import csv
import gc
import math
import numpy as np
import os
import pdb
import pickle
import statsmodels.api as sm
import time


import rpy2.robjects.packages as rpackages
from rpy2 import robjects

from scipy import stats
from scipy.integrate import simps
from scipy import interpolate

from src.helpers import save_dict, load_dict
from src.brc import BRC
from src.strategies import IronCondor
from src.quandl_data import get_btc_prices_2015_2022

from numpy.lib.function_base import insert

from statsmodels.nonparametric.kernel_regression import KernelReg as nadarajawatson

from sklearn.neighbors import KernelDensity

from pathlib import Path

def filter_sub(_df,  mindate, maxdate, tau):
    # Subset

    # Only calls, tau in [0, 0.25] and fix one day (bc looking at intra day here)
    # (_df['is_call'] == 1) & --> Dont need to filter for calls because that happens in the R script
    # Also need to consider put-call parity there
    sub = _df[(_df['date'] >= mindate) & (_df['date'] <= maxdate) & (_df['moneyness'] >= 0.5) & (_df['moneyness'] < 2) &(_df['mark_iv'] > 0)]# &

    if tau > 0:
        sub = sub[sub['tau'] == tau]

    nrows = sub.shape[0]
    if nrows == 0:
        raise(ValueError('Sub is empty'))
   
    sub['moneyness'] = round(sub['moneyness'], 3)
    sub['underlying_price'] = round(sub['underlying_price'], 2)

    sub = sub.drop_duplicates()
    
    print(sub.describe())

    if nrows > 50000:
        print('large df, subsetting')
        sub = sub.sample(10000)
        print(sub.describe())

    return sub

def prepare_confidence_band_data(df):

            # Old
                # Save for Confidence Band 
                    # Column 1 - Day
                    # Column 2 - Month
                    # Column 3 - Year
                    # Column 4 - Call Indicator
                    # Column 5 - Time Maturity in Days
                    # Column 6 - Strike
                    # Column 7 - IV
                    # Column 8 - Spot
                    # Column 9 - Risk Free Rate
                    # Column 10 - 
                    # Column 11 - Forward
                # 3	1	1997	0	49	3025	165.8	2861.706	0.031465	0.150026951	2863.26
                # Day Month Year Put Days Strike IV     Spot         Risk Free    Tau in Days (255)       Forward

    """
    # Mapping Table for Input

    # Column ... X
    # 1 - date
    # 2 - IV
    # 3 - 1 for Call, 0 for Put
    # 4 - Tau 
    # 5 - Strike
    # 6 - assigned meanwhile as put2call[,6]+ put2call[,7] -  put2call[,5]*exp(- mean(put2call[,8])* tau);
    # 7 - Spot
    # 8 - Probably Interest Rate
    # 9 - assigned meanwhile Moneyness: Spot/Strike
    """
    b = df[['date', 'mark_iv', 'putcall', 'tau', 'K', 'underlying_price', 'interest_rate', 'moneyness', 'instrument_price']]
    b['placeholder'] = 0
    b = b.reset_index(drop=True)
    # b['interest_rate'] = 0.00
    b['forward'] = 1

    # Reorder
    df_c = b[['date', 'mark_iv', 'putcall', 'tau', 'K', 'instrument_price', 'underlying_price', 'interest_rate', 'moneyness']]

    unique_maturities = df.days_to_maturity.unique()[0]
    fname = str('confidence_band_input_') + str(unique_maturities) + str('.csv')
    df_c.to_csv(fname, index = False)
    return fname

def bootstrap(conf_fname, phys_fname, rdate, tau, simmethod, r, out_dir, bw, lower, upper):
                            
    try:
        # MAKE SURE TO USE DIFFERENT 2ND INPUT FOR SP500!!
        moneyness, spd, epk, epk_lo, epk_up, logret, spd_logret, epk_logret, epk_logret_lo, epk_logret_up, epk_BS, tau, volas, spd_lo, spd_up, cdf_m, cdf_ret= r.bootstrap_epk(robjects.StrVector([conf_fname]), 
                                      robjects.StrVector([phys_fname]),
                                      robjects.StrVector(rdate),
                                      robjects.StrVector([out_dir]),
                                      robjects.FloatVector([bw]),
                                      robjects.FloatVector([lower]),
                                      robjects.FloatVector([upper])
                                     )
        # Can recover PD here as SPD/EPK
        spd_df = pd.DataFrame({'m': moneyness,
                               'spdy': spd,
                               'spdy_up': spd_up,
                               'spdy_dn': spd_lo,
                               'epk': epk,
                               'ret': logret,
                               'spd_ret': spd_logret,
                               'cb_ret_up': epk_logret_up,
                               'cb_ret_dn': epk_logret_lo,
                               'epk_ret': epk_logret,
                               'volatility': volas,
                               'epk_up': epk_up,
                               'epk_dn': epk_lo,
                               'cdf_m': cdf_m,
                               'cdf_ret': cdf_ret})

        return spd_df
    
    except Exception as e:
        print('Sim failed, proceeding with the next one... - ', simmethod)
        print('exception: ', e)
        
# Initiate R Objects
base = rpackages.importr("base") 
r = robjects.r
r.source('src/confidence_bands.R')
r.source('src/Q_Rookley_main_3.R')

# BTC prices
tdat = get_btc_prices_2015_2022()
tdat[::-1].to_csv('Data/BTC_USD_Quandl_2015_2022.csv', index = False)

bw_range = np.array([0.3])
r_bandwidth = bw_range[0]

simmethod = ['oldschool']

df = pd.read_csv('Data/20172022_processed_1_3_4.csv')
interest_rate_data = pd.read_csv('Data/IR_daily.csv')
interest_rate_data = interest_rate_data.rename(columns={'index': 'date','DTB3': 'interest_rate'})

df1=df[df['tau']==47]
dates=df1['date'].unique()
#df1=df[df['date']>=dates[9]]
#dates = df1['date'].unique()

dates1 = dates

error_i = []
for i0 in range(dates1.size):
    df2 = df1[df1['date']==dates1[i0]]
    
    curr_day = datetime.datetime.strptime(dates1[i0],'%Y-%m-%d')
    curr_day_starttime = curr_day.replace(hour = 0, minute = 0, second = 0, microsecond = 0)
    curr_day_endtime = curr_day.replace(hour = 23, minute = 59, second = 59, microsecond = 0)
    print(curr_day)

    df3 = pd.merge(df2, interest_rate_data, on='date', how='left')
    

    df3['interest_rate'] = 0
    df3['days_to_maturity'] = df3['tau']
    df3['tau'] = df3['tau']/365
    df3['underlying_price'] = df3['BTC_price']
    df3['instrument_price'] = df3['option_price']
    df3['putcall'][df3['putcall']=='C']=1
    df3['putcall'][df3['putcall']=='P']=0
    
    df_1 = df3[['date','BTC_price', 'K', 'interest_rate', 'tau', 'putcall', 'IV',  'moneyness',
                'days_to_maturity',  'underlying_price', 'instrument_price']]
    df_1['mark_iv']=df_1['IV']/100
    
    df_1.to_csv('Results/option_' + datetime.datetime.strftime(curr_day_starttime, '%Y%m%d') + '.csv')
    
    conf_fname = prepare_confidence_band_data(df_1)
    
    rdate = base.as_Date(curr_day.strftime('%Y-%m-%d'))
    rdate_f = base.format(rdate, '%Y-%m-%d')
    
    tau = df['tau'].unique()[0]
    
    spd_btc = bootstrap(conf_fname, 'Data/BTC_USD_Quandl_2015_2022.csv', rdate_f, tau, simmethod, r, 
                                     'Results/Rookley_Q_CB_1722_1_2_5_1_ttm47', r_bandwidth, df_1['moneyness'].min(), df_1['moneyness'].max())
    
    try :
        
        spd_btc.to_csv('Results/Rookley_Q_CB_1722_1_2_5_1_ttm47/btc_pk_'  + str(curr_day_starttime.__format__('%Y-%m-%d')) +'_bw_' + str(r_bandwidth) +  '.csv')
        
        print(spd_btc)
        
    except Exception as e:
        error_i.append(i0)

Using Macos Locale
2017-08-13 00:00:00


/var/folders/_7/3860x3293tn4h2dlrp5_t7_h0000gn/T/ipykernel_31174/3668656449.py:197: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['putcall'][df3['putcall']=='C']=1
/var/folders/_7/3860x3293tn4h2dlrp5_t7_h0000gn/T/ipykernel_31174/3668656449.py:198: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['putcall'][df3['putcall']=='P']=0
/var/folders/_7/3860x3293tn4h2dlrp5_t7_h0000gn/T/ipykernel_31174/3668656449.py:202: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pyda

[1] "bandwidth: 0.3"


R[write to console]: Error in (function (f, ts, currdate, out_dir, bandwidth, mon_low, mon_up)  : 
  Too few data points!

R[write to console]: In addition: 
R[write to console]: There were 50 or more warnings (use warnings() to see the first 50)
R[write to console]: 



Sim failed, proceeding with the next one... -  ['oldschool']
exception:  Error in (function (f, ts, currdate, out_dir, bandwidth, mon_low, mon_up)  : 
  Too few data points!

2017-11-12 00:00:00
[1] "bandwidth: 0.3"
[1] "test no arbitrage condition"


...

[1] 44.694713  4.722982 66.258718
               [,1]      [,2]         [,3]         [,4]
sigmas -0.758301920 1.0894207 -1.289866327  2.557415061
       -0.743563655 1.0805072 -1.272076919  2.542303792
       -0.729039456 1.0717176 -1.254397262  2.527218851
       -0.714723192 1.0630511 -1.236826974  2.512159651
       -0.700608994 1.0545068 -1.219365680  2.497125594
       -0.686691238 1.0460840 -1.202013011  2.482116065
       -0.672964529 1.0377818 -1.184768600  2.467130442
       -0.659423694 1.0295996 -1.167632089  2.452168087
       -0.646063767 1.0215364 -1.150603124  2.437228351
       -0.632879977 1.0135915 -1.133681360  2.422310576
       -0.619867740 1.0057641 -1.116866456  2.407414088
       -0.607022648 0.9980535 -1.100158077  2.392538207
       -0.594340463 0.9904589 -1.083555898  2.377682238
       -0.581817103 0.9829796 -1.067059598  2.362845479
       -0.569448640 0.9756146 -1.050668866  2.348027215
       -0.557231289 0.9683634 -1.034383396  2.333226724
       -0.5451

        0.385292291 0.7231467  0.188117291  0.321563711
        0.390012581 0.7244696  0.190237362  0.304214575
        0.394710693 0.7258067  0.192233776  0.286842264
        0.399386837 0.7271569  0.194106371  0.269447080
        0.404041216 0.7285195  0.195854985  0.252029315
        0.408674032 0.7298935  0.197479460  0.234589260
        0.413285484 0.7312782  0.198979639  0.217127200
        0.417875768 0.7326726  0.200355368  0.199643413
        0.422445078 0.7340758  0.201606496  0.182138173
        0.426993604 0.7354870  0.202732872  0.164611745
        0.431521534 0.7369052  0.203734349  0.147064392
        0.436029055 0.7383297  0.204610781  0.129496365
        0.440516349 0.7397595  0.205362022  0.111907913
        0.444983598 0.7411937  0.205987929  0.094299276
        0.449430979 0.7426315  0.206488361  0.076670685
        0.453858668 0.7440720  0.206863176  0.059022367
        0.458266839 0.7455143  0.207112236  0.041354538
        0.462655663 0.7469575  0.207235401  0.02

/var/folders/_7/3860x3293tn4h2dlrp5_t7_h0000gn/T/ipykernel_31174/3668656449.py:197: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['putcall'][df3['putcall']=='C']=1
/var/folders/_7/3860x3293tn4h2dlrp5_t7_h0000gn/T/ipykernel_31174/3668656449.py:198: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['putcall'][df3['putcall']=='P']=0
/var/folders/_7/3860x3293tn4h2dlrp5_t7_h0000gn/T/ipykernel_31174/3668656449.py:202: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pyda

[1] 44.694713  4.722982 66.258718
               [,1]      [,2]         [,3]          [,4]
sigmas -1.331066325 1.5688902 -2.816129111  5.2935960973
       -1.302405994 1.5473005 -2.773449240  5.2458577096
       -1.274544244 1.5260516 -2.731219688  5.1984808377
       -1.247437774 1.5051393 -2.689435792  5.1514627476
       -1.221046716 1.4845595 -2.648092912  5.1048006419
       -1.195334274 1.4643083 -2.607186438  5.0584916609
       -1.170266423 1.4443816 -2.566711789  5.0125328838
       -1.145811632 1.4247755 -2.526664415  4.9669213293
       -1.121940630 1.4054862 -2.487039798  4.9216539574
       -1.098626192 1.3865099 -2.447833454  4.8767276703
       -1.075842957 1.3678426 -2.409040932  4.8321393134
       -1.053567255 1.3494809 -2.370657821  4.7878856774
       -1.031776967 1.3314209 -2.332679746  4.7439634994
       -1.010451388 1.3136590 -2.295102369  4.7003694645
       -0.989571109 1.2961917 -2.257921395  4.6571002077
       -0.969117914 1.2790155 -2.221132570  4.61415231

        0.303775869 0.6037953  0.175009722  0.6032440372
        0.309428993 0.6051451  0.179522185  0.5767002578
        0.315050338 0.6065289  0.183833967  0.5501884254
        0.320640261 0.6079449  0.187945352  0.5237084080
        0.326199110 0.6093917  0.191856622  0.4972600971
        0.331727229 0.6108678  0.195568067  0.4708434081
        0.337224956 0.6123716  0.199079974  0.4444582806
        0.342692622 0.6139017  0.202392637  0.4181046782
        0.348130556 0.6154564  0.205506349  0.3917825889
        0.353539079 0.6170344  0.208421410  0.3654920252
        0.358918507 0.6186340  0.211138123  0.3392330243
        0.364269152 0.6202538  0.213656793  0.3130056481
        0.369591319 0.6218923  0.215977731  0.2868099837
        0.374885311 0.6235479  0.218101255  0.2606461431
        0.380151424 0.6252192  0.220027686  0.2345142638
        0.385389950 0.6269047  0.221757351  0.2084145085
        0.390601177 0.6286029  0.223290584  0.1823470654
        0.395785388 0.6303122  

In [4]:
print(error_i)

[0, 2, 3, 5, 6, 7, 8, 9, 10, 11, 12, 14, 15, 16, 18, 19]
